# ライブラリのインストール (必要な方だけ)

# ソースコード
## ディープラーニングモジュールのimport

In [122]:
# pytroch(ディープラーニングに必要なモジュール)　をインポートして利用できるようにする。
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

## 学習に必要なデータの作成

In [123]:
CONTEXT_SIZE = 2 # 文脈の情報
EMBEDDING_SIZE = 10 # 各語彙に割り当てるベクトルの次元数

`CONTEXT_SIZE`はどれだけの周辺単語を考慮するかを決定する変数。例えば4とすれば、ターゲットにした語彙の周辺４語彙を考慮して学習データが作成される  
`EMBEDDING_SIZE`は各語彙に割り当てるベクトルの次元数

In [124]:
sentences = """「 クリスティーヌ 。 お前 に は 隣国 の 貴族 に 嫁い で もらう 」 「 … … え ？ 」 私 は 驚い て 目 を 見開い た 。 目 の 前 に は 立派 な 口髭 を 生やし た 壮年 の 男 が 、 私 を 見下ろし て いる 。 その 周り に は 、 世 に も 美しい 顔立ち の し た 金髪 碧眼 の 青年 や 、 眼鏡 を 掛け た クール な 美 男子 や 、 その 顔 に あどけな さ を 残す 可愛らしい 美少年 が 、 総じて 私 を 睨ん で い た 。 そして もう 一人 、 素朴 な 可愛らし さ が ある 少女 が 、 困っ た 顔 で 私 を 見 て いる 。 私 は クリスティーヌ ・ ブラン シャーネ 。 王国 有数 の ブラン シャーネ 公爵 の 一人娘 で 、 小さい 頃 から 蝶 よ 花 よ と 育て られ 、 それ は それ は 傲慢 な 娘 に 育っ た 。 生まれ た 時 から 、 望ん だ もの は 全部 与え られる 。 そして それ を 当然 だ と 思っ て い た 。"""

上記の半角スペースで区切られた文章を整形していき、学習データを作成する。  
実際の収集してきたデータはスペースで区切られていないが、これには形態素解析ツールを利用するとこのような形で分割できる。

In [125]:
vocab_sequence = sentences.split() # 半角スペースで文字を区切る
vocab_set = set(vocab_sequence) # set コンストラクタにリストを渡して要素の重複をなくした
word2id = {word: id for id, word in enumerate(vocab_set)}# word → idを得るための辞書の作成
id2word = {id: word for word, id in word2id.items()} # id →　wordを得るための辞書の作成
train_data = [
  (
    [word2id[vocab_sequence[target - CONTEXT_SIZE + j]] for j in range(CONTEXT_SIZE * 2 + 1) if target - CONTEXT_SIZE + j != target], # ターゲットに対するコンテキストデータをまとめます。
    word2id[vocab_sequence[target]] # ラベルデータ(正解)
  )
  for target in range(CONTEXT_SIZE, len(vocab_sequence) - CONTEXT_SIZE - 1)
] # 学習データの作成

In [126]:
train_data[:3] # 0番目から2番目までの学習データを確認

[([10, 41, 4, 16], 0), ([41, 0, 16, 95], 4), ([0, 4, 95, 59], 16)]

In [127]:
train_data[-1]

([53, 40, 21, 6], 32)

これで学習データの準備が完了

## 学習モデルを定義
CBOWとskip-gramがあると説明しましたが、ここではより単純なモデルであるCBOWのモデルを定義します。 
最低限必要な実装は`forward`のみです。データをどのように処理していくのかをここに記述します。


forwardに入力する行列x（テンソル）の構造の変化
```
(N × CONTEXT_SIZE)   
→embed→ (CONTEXT_SIZE × embedding_dim)  
→view((1,-1))→ ((CONTEXT_SIZE * embedding_dim))  
→linearA(x)→ (hidden_dim)
→F.relu(x)→ (hidden_dim)
→linearB(x)→　(vocab_size)
→F.log_softmax(x, dim=1)→　(vocab_size)
```

In [128]:
class CBOW(nn.Module):
  def __init__(self, vocab_size, embedding_dim, hidden_dim):
    super(CBOW, self).__init__()
    self.embed = nn.Embedding(vocab_size, embedding_dim) # embedding により語彙IDをそれぞれに対応するベクトルに変換する
    self.linearA = nn.Linear(CONTEXT_SIZE * 2 * embedding_dim, hidden_dim) # 全結合層。CONTEXT_SIZE個の学習データをまとめて入力したいので、入力次元数をCONTEXT_SIZE * 2 * embedding_dimのようにしている。
    self.linearB = nn.Linear(hidden_dim, vocab_size)
    pass

  def forward(self, x): # 順伝播を定義します。
    # x は N × CONTEXT_SIZEのtensor.　ここで、Nはバッチデータサイズ
    x = self.embed(x).view((1, -1))
    x = self.linearA(x)
    x = F.relu(x)
    x = self.linearB(x)
    return F.log_softmax(x, dim=1)


## トレーニングの実行

In [129]:
vocab_size = len(word2id) # 語彙数
model = CBOW(vocab_size, EMBEDDING_SIZE, 128) # クラスからインスタンスを作成
nlloss_function = nn.NLLLoss() #　使用する損失関数の宣言
optimizer = optim.SGD(model.parameters(), lr=0.01) # modelのパラメータをSGD最適化モジュールに設定
epoch_num = 100 # 学習回数

for epoch in range(epoch_num):
  loss_sum = 0
  for contexts, target in train_data:
    model.zero_grad()
    contexts = torch.tensor(contexts, dtype=torch.long)
    labels = torch.tensor([target], dtype=torch.long)

    out = model(contexts) # モデルにデータを入力

    loss = nlloss_function(out, labels) # 正解ラベルとの誤差を計算
    loss.backward()
    optimizer.step() # パラメータの修正を行う
    loss_sum += loss.item() # 損失を加算
  if (epoch+1) % 10 == 0: # 10回ごとに損失の平均値を出力
    print('loss', loss_sum / epoch_num)


loss 3.496219913363457
loss 0.44183608192950485
loss 0.15740457333624364
loss 0.1010095592495054
loss 0.07747589786187746
loss 0.0642921561805997
loss 0.05572149791172706
loss 0.04965983530622907
loss 0.04513972062733956
loss 0.04160438053542748


損失が徐々に下がっていっている。つまり、モデルが学習データによってトレーニングされていっている。

## 学習したモデルの挙動を確認する
学習データの一部「私を見下ろしている」を利用して、ちゃんと学習したものに答えられるのかどうかを確認してみる。

In [130]:
parsed = "私 を 見下ろし て いる".split() # 学習データ中のデータ
test_data = [word2id[word] for word in parsed]
test_data = test_data[:CONTEXT_SIZE] + test_data[CONTEXT_SIZE+1:CONTEXT_SIZE + 1 + CONTEXT_SIZE]
print('入力するデータ:', [id2word[word_id] for word_id in test_data], 'をidに変換したもの')

入力するデータ: ['私', 'を', 'て', 'いる'] をidに変換したもの


入力するデータは上記の通りになる。

In [131]:
test_data = torch.tensor(test_data)
p = model(test_data) # テストデータを入れた場合の語彙を計算する。

In [132]:
for idx, i in enumerate(torch.topk(p, 5, largest=True).indices[0]): # torch.topkで降順に５つの値を取得し、そのインデックスをforで回している。
  print(idx + 1, id2word[i.item()])

1 見
2 見下ろし
3 驚い
4 睨ん
5 生やし


本当の正解は「見下ろし」だが、近い意味を持つ「見」が一番上に来ていることがわかる。  
つまり、「見下ろし」と「見」に割り当てられたベクトルがトレーニングによって近いベクトルになったというわけである。

なお、この程度の学習データ量で、このような好ましい結果が得られることの方が稀である  
実際の利用に耐えうるようなベクトル表現を学習させようと思うなら、もっとたくさんの学習データが必要であることを留意して欲しい。
